In [5]:
#stolen from https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277
import os
import sys
import random
from collections import OrderedDict
import itertools
import math
import logging
import json
import re

import numpy as np
import pandas as pd


import utils
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input, AlphaDropout, Activation
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adamax, Adam, SGD
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'train/'
#TEST_PATH = 'test/'

In [17]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [8]:
# initializes the training images and target images (masks) as arrays of zeros
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
#gets and resizes the images each containing several nuclei
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    #X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    #gets and resizes the masks that contain the individual segmented nuclei
    #the targets for the model
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (256, 256), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
    
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
sizes_test = []
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')


  6%|▌         | 34/607 [00:08<02:14,  4.25it/s]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 28%|██▊       | 168/607 [00:33<01:27,  5.04it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'train/.ipynb_checkpoints/images/.ipynb_checkpoints.png'

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.1, random_state=8011)

print(len(x_train), len(y_train), len(x_valid), len(y_valid))
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

603 603 67 67
(603, 256, 256, 3) (603, 256, 256, 1) (67, 256, 256, 3) (67, 256, 256, 1)


In [22]:
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.losses import binary_crossentropy

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [23]:

nb_filters_reduction_factor = 8

# Build InceptionResnetV2 stem block connected to U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, 3))
s = Lambda(lambda x: x/255) (inputs)

x = Conv2D(32//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(s)
x = Conv2D(32//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)
x = Conv2D(64//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)

a = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)

b = Conv2D(96//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)
x = concatenate([a, b], axis=3)

a = Conv2D(64//nb_filters_reduction_factor, (1, 1), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)
a = Conv2D(96//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(a)
b = Conv2D(64//nb_filters_reduction_factor, (1, 1), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)
b = Conv2D(64//nb_filters_reduction_factor, (7, 1), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(b)
b = Conv2D(64//nb_filters_reduction_factor, (1, 7), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(b)
b = Conv2D(96//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(b)

x = concatenate([a, b], axis=3)

a = Conv2D(192//nb_filters_reduction_factor, (3, 3), subsample=(1, 1), activation='elu',
                      init='he_normal', padding='same')(x)

b = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
x = concatenate([a, b], axis=3)

x = Activation('elu')(x)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (x)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer=Adamax(lr=0.002), loss='binary_crossentropy', metrics=[mean_iou])

model.summary()

"""
#from https://github.com/flyyufelix/cnn_finetune/blob/master/resnet_101.py

from keras.layers.core import Layer
from keras.engine import InputSpec
from keras import backend as K
try:
    from keras import initializations
except ImportError:
    from keras import initializers as initializations
    
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K

from sklearn.metrics import log_loss

import sys
sys.setrecursionlimit(3000)

class Scale(Layer):
    '''Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases learned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        # Compatibility with TensorFlow >= 1.0.0
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        #self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        #self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = merge([x, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Convolution2D(nb_filter1, 1, 1, subsample=strides,
                      name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Convolution2D(nb_filter2, kernel_size, kernel_size,
                      name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Convolution2D(nb_filter3, 1, 1, name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
                             name=conv_name_base + '1', bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x
"""
    
"""    
  Resnet 101 Model for Keras
    Model Schema and layer naming follow that of the original Caffe implementation
    https://github.com/KaimingHe/deep-residual-networks
    ImageNet Pretrained Weights 
    Theano: https://drive.google.com/file/d/0Byy2AcGyEVxfdUV1MHJhelpnSG8/view?usp=sharing
    TensorFlow: https://drive.google.com/file/d/0Byy2AcGyEVxfTmRRVmpGWDczaXM/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
"""
  
"""
eps = 1.1e-5
bn_axis = 3
img_input = Input(shape=(256, 256, 3), name='data')

x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1', use_bias=False)(x)
x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
x = Scale(axis=bn_axis, name='scale_conv1')(x)
x = Activation('relu', name='conv1_relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
for i in range(1,4):
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
for i in range(1,23):
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

#x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
#x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
#x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

#x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
#x_fc = Flatten()(x_fc)
#x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

model = Model(img_input, x)

# Use pre-trained weights for Tensorflow backend
weights_path = 'resnet101_weights_tf.h5'

model.load_weights(weights_path, by_name=True)

# Truncate and replace softmax layer for transfer learning
# Cannot use model.layers.pop() since model is not of Sequential() type
# The method below works since pre-trained weights are stored in layers but not in the model
#x_newfc = AveragePooling2D((7, 7), name='avg_pool')(x)
#x_newfc = Flatten()(x_newfc)
x_newfc = Conv2D(1, (1, 1), activation='sigmoid')(x)

model = Model(img_input, x_newfc)

# Learning rate is changed to 0.001
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=[mean_iou])
model.summary()
"""


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="elu", padding="same", strides=(1, 1), kernel_initializer="he_normal")`
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="elu", padding="same", strides=(1, 1), kernel_initializer="he_normal")`
  # This is added back by InteractiveShellApp.init_path()
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="elu", padding="same", strides=(1, 1), kernel_initializer="he_normal")`
  del sys.path[0]
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 4)  112         lambda_3[0][0]                   
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 256, 256, 4)  148         conv2d_61[0][0]                  
__________________________________________________________________________________________________
conv2d_63 

"\neps = 1.1e-5\nbn_axis = 3\nimg_input = Input(shape=(256, 256, 3), name='data')\n\nx = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)\nx = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1', use_bias=False)(x)\nx = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)\nx = Scale(axis=bn_axis, name='scale_conv1')(x)\nx = Activation('relu', name='conv1_relu')(x)\nx = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)\n\nx = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))\nx = identity_block(x, 3, [64, 64, 256], stage=2, block='b')\nx = identity_block(x, 3, [64, 64, 256], stage=2, block='c')\n\nx = conv_block(x, 3, [128, 128, 512], stage=3, block='a')\nfor i in range(1,4):\n  x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))\n\nx = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')\nfor i in range(1,23):\n  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))\n\n#x = conv_block(x, 3, [512, 512, 20

In [24]:
from keras.preprocessing.image import ImageDataGenerator



data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=90.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
image_datagen.fit(x_train, seed=7)
mask_datagen.fit(y_train, seed=7)
image_generator = image_datagen.flow(x_train, batch_size=32, seed=7)
mask_generator = mask_datagen.flow(y_train, batch_size=32, seed=7)
train_generator = zip(image_generator, mask_generator)

val_gen_args = dict()
image_datagen_val = ImageDataGenerator(**val_gen_args)
mask_datagen_val = ImageDataGenerator(**val_gen_args)
image_datagen_val.fit(x_valid, seed=7)
mask_datagen_val.fit(y_valid, seed=7)
image_generator_val = image_datagen_val.flow(x_valid, batch_size=32, seed=7)
mask_generator_val = mask_datagen_val.flow(y_valid, batch_size=32, seed=7)
valid_generator=zip(image_generator_val, mask_generator_val)


In [25]:
# Fit model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
#lr_reduce = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience=5, min_lr = 0.00001)
#earlystop = EarlyStopping(patience=30, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
#model.fit_generator(train_generator, steps_per_epoch=len(x_train)/6, epochs=50,validation_data=valid_generator, validation_steps=len(xval)/batch_size)
    
model.fit(x_train, y_train, batch_size=16, epochs = 100, validation_data=(x_valid, y_valid), callbacks = [checkpoint])


Train on 603 samples, validate on 67 samples
Epoch 1/100
592/603 [============================>.] - ETA: 0s - loss: 0.3721 - mean_iou: 0.4155
Epoch 00001: val_loss improved from inf to 0.25759, saving model to best_model.h5
603/603 [==============================] - 41s 68ms/step - loss: 0.3698 - mean_iou: 0.4160 - val_loss: 0.2576 - val_mean_iou: 0.4524
Epoch 2/100
592/603 [============================>.] - ETA: 0s - loss: 0.1939 - mean_iou: 0.5135
Epoch 00002: val_loss improved from 0.25759 to 0.17971, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.1933 - mean_iou: 0.5142 - val_loss: 0.1797 - val_mean_iou: 0.5593
Epoch 3/100
592/603 [============================>.] - ETA: 0s - loss: 0.1551 - mean_iou: 0.5889
Epoch 00003: val_loss improved from 0.17971 to 0.14575, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.1543 - mean_iou: 0.5893 - val_loss: 0.1457 - val_mean_iou: 0.6161
Epoch 4/100

603/603 [==============================] - 33s 55ms/step - loss: 0.0758 - mean_iou: 0.7985 - val_loss: 0.0859 - val_mean_iou: 0.7994
Epoch 29/100
592/603 [============================>.] - ETA: 0s - loss: 0.0724 - mean_iou: 0.8003
Epoch 00029: val_loss improved from 0.08592 to 0.08471, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.0725 - mean_iou: 0.8004 - val_loss: 0.0847 - val_mean_iou: 0.8013
Epoch 30/100
592/603 [============================>.] - ETA: 0s - loss: 0.0740 - mean_iou: 0.8022
Epoch 00030: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0741 - mean_iou: 0.8022 - val_loss: 0.0871 - val_mean_iou: 0.8030
Epoch 31/100
592/603 [============================>.] - ETA: 0s - loss: 0.0721 - mean_iou: 0.8039
Epoch 00031: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0720 - mean_iou: 0.8039 - val_loss: 0.0881 - val_mean_iou: 0.8048
Epoch 32/100

Epoch 58/100
592/603 [============================>.] - ETA: 0s - loss: 0.0538 - mean_iou: 0.8357
Epoch 00058: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0537 - mean_iou: 0.8357 - val_loss: 0.0900 - val_mean_iou: 0.8361
Epoch 59/100
592/603 [============================>.] - ETA: 0s - loss: 0.0518 - mean_iou: 0.8366
Epoch 00059: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0523 - mean_iou: 0.8366 - val_loss: 0.0836 - val_mean_iou: 0.8371
Epoch 60/100
592/603 [============================>.] - ETA: 0s - loss: 0.0534 - mean_iou: 0.8375
Epoch 00060: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0538 - mean_iou: 0.8375 - val_loss: 0.0823 - val_mean_iou: 0.8379
Epoch 61/100
592/603 [============================>.] - ETA: 0s - loss: 0.0531 - mean_iou: 0.8383
Epoch 00061: val_loss did not improve
603/603 [==============================] - 33s 54ms/step 

Epoch 89/100
592/603 [============================>.] - ETA: 0s - loss: 0.0386 - mean_iou: 0.8586
Epoch 00089: val_loss did not improve
603/603 [==============================] - 34s 56ms/step - loss: 0.0388 - mean_iou: 0.8586 - val_loss: 0.0970 - val_mean_iou: 0.8589
Epoch 90/100
592/603 [============================>.] - ETA: 0s - loss: 0.0389 - mean_iou: 0.8592
Epoch 00090: val_loss did not improve
603/603 [==============================] - 34s 56ms/step - loss: 0.0388 - mean_iou: 0.8592 - val_loss: 0.0974 - val_mean_iou: 0.8595
Epoch 91/100
592/603 [============================>.] - ETA: 0s - loss: 0.0381 - mean_iou: 0.8598
Epoch 00091: val_loss did not improve
603/603 [==============================] - 34s 56ms/step - loss: 0.0382 - mean_iou: 0.8598 - val_loss: 0.0983 - val_mean_iou: 0.8601
Epoch 92/100
592/603 [============================>.] - ETA: 0s - loss: 0.0380 - mean_iou: 0.8604
Epoch 00092: val_loss did not improve
603/603 [==============================] - 34s 56ms/step 

In [26]:
model.load_weights('best_model.h5')

In [27]:
lr_reduce = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience=5, min_lr = 0.0001)
earlystop = EarlyStopping(patience=30, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
model.compile(optimizer=SGD(lr=0.002), loss='binary_crossentropy', metrics=[mean_iou])
model.fit(x_train, y_train, batch_size=16, epochs = 50, validation_data=(x_valid, y_valid), callbacks = [checkpoint])

Train on 603 samples, validate on 67 samples
Epoch 1/50
592/603 [============================>.] - ETA: 0s - loss: 0.0586 - mean_iou: 0.8479
Epoch 00001: val_loss improved from inf to 0.07997, saving model to best_model.h5
603/603 [==============================] - 37s 61ms/step - loss: 0.0587 - mean_iou: 0.8483 - val_loss: 0.0800 - val_mean_iou: 0.8699
Epoch 2/50
592/603 [============================>.] - ETA: 0s - loss: 0.0581 - mean_iou: 0.8721
Epoch 00002: val_loss improved from 0.07997 to 0.07987, saving model to best_model.h5
603/603 [==============================] - 33s 56ms/step - loss: 0.0582 - mean_iou: 0.8721 - val_loss: 0.0799 - val_mean_iou: 0.8724
Epoch 3/50
592/603 [============================>.] - ETA: 0s - loss: 0.0579 - mean_iou: 0.8729
Epoch 00003: val_loss improved from 0.07987 to 0.07983, saving model to best_model.h5
603/603 [==============================] - 34s 56ms/step - loss: 0.0580 - mean_iou: 0.8729 - val_loss: 0.0798 - val_mean_iou: 0.8738
Epoch 4/50
592

592/603 [============================>.] - ETA: 0s - loss: 0.0566 - mean_iou: 0.8805
Epoch 00030: val_loss did not improve
603/603 [==============================] - 32s 54ms/step - loss: 0.0569 - mean_iou: 0.8805 - val_loss: 0.0797 - val_mean_iou: 0.8805
Epoch 31/50
592/603 [============================>.] - ETA: 0s - loss: 0.0564 - mean_iou: 0.8805
Epoch 00031: val_loss improved from 0.07955 to 0.07948, saving model to best_model.h5
603/603 [==============================] - 33s 54ms/step - loss: 0.0569 - mean_iou: 0.8805 - val_loss: 0.0795 - val_mean_iou: 0.8806
Epoch 32/50
592/603 [============================>.] - ETA: 0s - loss: 0.0564 - mean_iou: 0.8806
Epoch 00032: val_loss did not improve
603/603 [==============================] - 32s 54ms/step - loss: 0.0569 - mean_iou: 0.8806 - val_loss: 0.0797 - val_mean_iou: 0.8806
Epoch 33/50
592/603 [============================>.] - ETA: 0s - loss: 0.0569 - mean_iou: 0.8807
Epoch 00033: val_loss did not improve
603/603 [================

In [28]:
model.load_weights('best_model.h5')

In [29]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
model.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=[mean_iou])
model.fit(x_train, y_train, batch_size=16, epochs = 50, validation_data=(x_valid, y_valid), callbacks = [checkpoint])

Train on 603 samples, validate on 67 samples
Epoch 1/50
592/603 [============================>.] - ETA: 0s - loss: 0.1704 - mean_iou: 0.7220
Epoch 00001: val_loss improved from inf to 0.14717, saving model to best_model.h5
603/603 [==============================] - 38s 63ms/step - loss: 0.1704 - mean_iou: 0.7219 - val_loss: 0.1472 - val_mean_iou: 0.7166
Epoch 2/50
592/603 [============================>.] - ETA: 0s - loss: 0.1519 - mean_iou: 0.7175
Epoch 00002: val_loss improved from 0.14717 to 0.13945, saving model to best_model.h5
603/603 [==============================] - 33s 56ms/step - loss: 0.1518 - mean_iou: 0.7176 - val_loss: 0.1394 - val_mean_iou: 0.7197
Epoch 3/50
592/603 [============================>.] - ETA: 0s - loss: 0.1214 - mean_iou: 0.7295
Epoch 00003: val_loss improved from 0.13945 to 0.12283, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.1221 - mean_iou: 0.7297 - val_loss: 0.1228 - val_mean_iou: 0.7372
Epoch 4/50
592

603/603 [==============================] - 33s 55ms/step - loss: 0.0703 - mean_iou: 0.8251 - val_loss: 0.0827 - val_mean_iou: 0.8257
Epoch 30/50
592/603 [============================>.] - ETA: 0s - loss: 0.0700 - mean_iou: 0.8264
Epoch 00030: val_loss did not improve
603/603 [==============================] - 33s 55ms/step - loss: 0.0707 - mean_iou: 0.8264 - val_loss: 0.0868 - val_mean_iou: 0.8269
Epoch 31/50
592/603 [============================>.] - ETA: 0s - loss: 0.0706 - mean_iou: 0.8274
Epoch 00031: val_loss improved from 0.08273 to 0.08089, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.0706 - mean_iou: 0.8274 - val_loss: 0.0809 - val_mean_iou: 0.8279
Epoch 32/50
592/603 [============================>.] - ETA: 0s - loss: 0.0677 - mean_iou: 0.8286
Epoch 00032: val_loss did not improve
603/603 [==============================] - 33s 55ms/step - loss: 0.0676 - mean_iou: 0.8286 - val_loss: 0.0850 - val_mean_iou: 0.8291
Epoch 33/50
592

In [30]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
model.compile(optimizer=Adamax(lr=0.001), loss='binary_crossentropy', metrics=[mean_iou])
model.fit(x_train, y_train, batch_size=16, epochs = 50, validation_data=(x_valid, y_valid), callbacks = [checkpoint])

Train on 603 samples, validate on 67 samples
Epoch 1/50
592/603 [============================>.] - ETA: 0s - loss: 0.0620 - mean_iou: 0.8545
Epoch 00001: val_loss improved from inf to 0.07860, saving model to best_model.h5
603/603 [==============================] - 38s 63ms/step - loss: 0.0617 - mean_iou: 0.8549 - val_loss: 0.0786 - val_mean_iou: 0.8757
Epoch 2/50
592/603 [============================>.] - ETA: 0s - loss: 0.0578 - mean_iou: 0.8772
Epoch 00002: val_loss improved from 0.07860 to 0.07756, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.0585 - mean_iou: 0.8773 - val_loss: 0.0776 - val_mean_iou: 0.8777
Epoch 3/50
592/603 [============================>.] - ETA: 0s - loss: 0.0584 - mean_iou: 0.8792
Epoch 00003: val_loss improved from 0.07756 to 0.07751, saving model to best_model.h5
603/603 [==============================] - 33s 55ms/step - loss: 0.0583 - mean_iou: 0.8792 - val_loss: 0.0775 - val_mean_iou: 0.8791
Epoch 4/50
592

Epoch 31/50
592/603 [============================>.] - ETA: 0s - loss: 0.0511 - mean_iou: 0.8879
Epoch 00031: val_loss did not improve
603/603 [==============================] - 33s 55ms/step - loss: 0.0508 - mean_iou: 0.8879 - val_loss: 0.0775 - val_mean_iou: 0.8880
Epoch 32/50
592/603 [============================>.] - ETA: 0s - loss: 0.0509 - mean_iou: 0.8881
Epoch 00032: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - loss: 0.0506 - mean_iou: 0.8881 - val_loss: 0.0790 - val_mean_iou: 0.8882
Epoch 33/50
592/603 [============================>.] - ETA: 0s - loss: 0.0513 - mean_iou: 0.8883
Epoch 00033: val_loss did not improve
603/603 [==============================] - 33s 55ms/step - loss: 0.0510 - mean_iou: 0.8883 - val_loss: 0.0776 - val_mean_iou: 0.8884
Epoch 34/50
592/603 [============================>.] - ETA: 0s - loss: 0.0506 - mean_iou: 0.8885
Epoch 00034: val_loss did not improve
603/603 [==============================] - 33s 54ms/step - lo

In [31]:
# Predict on train, val and test
model = load_model('best_model.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(x_train, verbose=1)
preds_val = model.predict(x_valid, verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

65/65 [==============================] - 1s 18ms/step


In [32]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

In [33]:
# Create submission DataFrame

sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.groupby('ImageId').ngroup()
print(len(sub))
sub.to_csv('11th_kaggle_submission_ever.csv', index=False)#scored 0.341




2623
